# EXP movie example 

## Using EXP coefficients, the EXP config file to construct a basis, and slices from the FieldGenerator

Read in config from EXP yaml file and render a movie from the coefficient file.   This notebook will create the movie for either the disk or the halo by changing the component variable from 'dark' to 'star' and vice versa.

## Begin with the usual imports

You may need to append the pyEXP location to your Python path, depending on your installation.

In [12]:
import os
import copy
import yaml
# sys.path.append('/my/path/to/site-packages')
import pyEXP
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors
from os.path import exists

plt.rcParams['figure.figsize'] = [12, 9]

## Configure the basis and component

At minimum, you need to
- Define the EXP config file to import the config.   
- Define the target component.

The rest of the notebook should then run without changes.  

Two optional parameters:
- The half size of each axis
- Number of pixels along each axis

In [13]:
# key parameters
exp_config = '../config.yml'
component  = 'star disk'  # You can make the halo (disk) movie by changing this to 'dark' ('star')
comp_name  = 'star'       # No spaces for movie generation

# options
size       = 0.04
npix       = 200

## Switch to the working directory
I like to be explicit about my working directory but you don't need to do this here.  It would be sufficient to simply pass the full path to the coefficient factory below or launch the notebook from a working directory.

In [14]:
os.chdir('/data/Nbody/NewNbody/new')

## Read the EXP config file, generate the basis from the config, and get the run tag for convenience

In [ ]:
# Open and read the yaml file
#
with open(exp_config, 'r') as f:
    yaml_db = yaml.load(f, Loader=yaml.FullLoader)
    

In order to make sure that your YAML configuration file matches the cache file from a previously
run simulation or computed coefficient set, the `BasisFactory` requires that you specify the cache
file name.  EXP will create that name on the fly.  So, we need to add the cache file name to the
YAML configuration by hand.  This may seem a bit awkward, but this level of intentional specification
will help prevent confusion later.

Specifically in the lines below, we iterate through the `Components` stanza and add the name cache
to the configuration.  Here, that named cache is `.eof.cache.run0`.

In [27]:
# Grab both star and dark, although I'm mostly interested in star at this point
#
for v in yaml_db['Components']:
    if v['name'] == component:
        # Add cache file
        v['force']['parameters']['eof_file'] = '.eof.cache.run0'
        # Get the config
        config = yaml.dump(v['force'])
        
print(config)
 

id: cylinder
parameters:
  acyl: 0.01
  ashift: 0
  density: true
  eof_file: .eof.cache.run0
  hcyl: 0.001
  lmaxfid: 32
  logr: false
  mmax: 6
  ncylnx: 128
  ncylny: 64
  ncylodd: 3
  nmax: 12
  nmaxfid: 32
  npca: 100
  pcadiag: true
  pcavtk: true
  pnum: 0
  rnum: 200
  self_consistent: true
  tk_type: Hall
  tnum: 80
  vflag: 0



## Check ranges in fields

In [28]:
       
# Construct the basis instance
#
basis = pyEXP.basis.Basis.factory(config)

# Get the runtag
#
runtag = yaml_db['Global']['runtag']
print("\nRuntag from {} is: {}".format(exp_config, runtag))
coeffile = 'outcoef.{}.{}'.format(component, runtag)
print("\nCoef file is:", coeffile)


Runtag from ../config.yml is: run0

Coef file is: outcoef.star disk.run0
---- EmpCylSL::ReadH5Cache: read <.eof.cache.run0>
---- EmpCylSL::read_cache: table forwarded to all processes


## Check ranges in fields

## Read the coefficients

In [29]:
coefs = pyEXP.coefs.Coefs.factory(coeffile)

## Set the output field grid and render the slices

In [30]:
import time
start = time.time()


times = coefs.Times()
pmin  = [-size, -size, 0.0]
pmax  = [ size,  size, 0.0]
grid  = [ npix,  npix,   0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

print('Created fields instance')

surfaces = fields.slices(basis, coefs)
print("Elapsed time:", time.time() - start)

Created fields instance
Elapsed time: 6.028102159500122


## Check ranges in fields

In [31]:
data = surfaces[times[-1]]
for v in data:
    print('{:20s}:  min={:10.2e}  max={:10.2e}  shape={}'.format(v, np.min(data[v]), np.max(data[v]), data[v].shape))

azi force           :  min= -3.82e-01  max=  3.17e-01  shape=(200, 200)
dens                :  min=  7.78e+01  max=  5.27e+05  shape=(200, 200)
dens m=0            :  min=  9.60e+01  max=  5.08e+05  shape=(200, 200)
mer force           :  min= -5.94e+01  max=  1.21e-01  shape=(200, 200)
potl                :  min= -2.71e+00  max= -4.49e-01  shape=(200, 200)
potl m=0            :  min= -2.66e+00  max= -4.62e-01  shape=(200, 200)
rad force           :  min= -1.33e+02  max=  4.16e+01  shape=(200, 200)


## Make a movie frames

In [32]:
start = time.time()

# Get the shape
keys = list(surfaces.keys())
nx = surfaces[keys[0]]['dens'].shape[0]
ny = surfaces[keys[0]]['dens'].shape[1]

# Make the mesh
x = np.linspace(pmin[0], pmax[0], nx)
y = np.linspace(pmin[1], pmax[1], ny)
xv, yv = np.meshgrid(x, y)

plt.rcParams.update({'font.size': 22})

# Fix the contour levels to prevent jitter in the movie
cbar1 = 10**np.arange(0.0, 4.01, 0.1)
cbar2 = 10**np.arange(0.0, 4.01, 0.4)

# Frame counter
icnt = 0
cmap = copy.copy(plt.colormaps['viridis'])

N = cmap.N
cmap.set_under(cmap(1))
cmap.set_over(cmap(N-1))

# Iterate through the keys
for v in keys:
    fig, ax = plt.subplots(1, 1, figsize=(24, 20))
    
    mat = surfaces[v]['dens']
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            if mat[i, j] < 1.0: mat[i, j] = 1.0
            if mat[i, j] > 10000.0: mat[i, j] = 10000.0
            
    cont1 = ax.contour(xv, yv, mat.transpose(), cbar2, colors='k')
    # You can label the contours inline by uncommenting the next two lines...
    # ax[0].clabel(cont1, fontsize=9, inline=True)
    # cont2 = ax.contourf(xv, yv, surfaces[v]['d'].transpose(), cbar2, vmin=cbar2[0], vmax=cbar2[-1])
    cont2 = ax.contourf(xv, yv, mat.transpose(), cbar1, locator=ticker.LogLocator())
    plt.colorbar(cont2, ax=ax)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title('T={:4.3f}'.format(v))
    
    fig.savefig('{}_movie_{}_{:04d}.png'.format(comp_name, runtag, icnt), dpi=75)
    plt.close()

    icnt += 1
    
print("Elapsed time:", time.time() - start)

Elapsed time: 41.79801082611084


## Make a mp4 file from the frames using ffmpeg

This only work if you have 'ffmpeg' installed, of course ...

In [10]:
os.system('ffmpeg -y -i \'{0}_movie_{1}_%04d.png\' movie_{0}_{1}.mp4'.format(comp_name, runtag))

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

0

### Preview the new movie

In [11]:
from IPython.display import Video
Video('movie_{0}_{1}.mp4'.format(comp_name, runtag), embed=True, width=800)